In [1]:
import jieba
from zhon import hanzi as zh_hanzi
import srt

from cedict_utils.cedict import CedictParser

from dragonmapper import transcriptions
from dragonmapper import hanzi

from IPython.display import HTML, display, Markdown, clear_output
# display(HTML('''<style>
#     .widget-label { max-width: 5ex !important; }
# </style>'''))

import ipywidgets as widgets

from pydub import AudioSegment

import cv2

import json
import urllib.request

import matplotlib.pyplot as plt

import numpy as np
import io

from PIL import Image

from pathlib import Path

import hashlib

from sys import platform

import datetime
import math

In [6]:
class Params:
#     episode = "beauty"
#     episode = "stereotypes"
#     episode = "livestreaming"
#     episode = "lying_flat"
#     episode = "ukraine_russia"
#     is_video = False
#     episode = "meteor_garden_s01e01"
#     episode = "studying_in_us"
    episode = "birthrate"
    is_video = True

In [7]:
DECK_MAP = {
    "beauty" : "Beauty Standards",
    "stereotypes" : "Chinese Stereotypes",
    "livestreaming" : "Livestreaming",
    "lying_flat" : "Lying Flat",
    "ukraine_russia" : "Ukraine-Russia",
    "meteor_garden_s01e01": "Meteor Garden",
    "studying_in_us": "Studying in US",
    "birthrate" : "Birthrate",
}

In [8]:
# Make a set so faster searching
PUNCTUATION = set(zh_hanzi.punctuation)

In [9]:
parser = CedictParser()
entries = parser.parse()

CEDICT = dict()
for e in entries:
    CEDICT[e.simplified] = e

In [10]:
if not Params.is_video:
    audio_file = AudioSegment.from_file(Params.episode + ".mp3")
if Params.is_video:
    audio_file = AudioSegment.from_file(Params.episode + ".mp4")
    vidcap = cv2.VideoCapture(Params.episode + ".mp4")

In [11]:
# Anki connect boilerplate

def request(action, **params):
    return {'action': action, 'params': params, 'version': 6}

def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
#     print(type(requestJson))
#     print(requestJson)
#     print(urllib.request.Request('http://localhost:8765', requestJson))
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))
    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')
    if 'error' not in response:
        raise Exception('response is missing required error field')
    if 'result' not in response:
        raise Exception('response is missing required result field')
    if response['error'] is not None:
        raise Exception(response['error'])
    return response['result']

# invoke('createDeck', deck='test1')
# result = invoke('deckNames')
# print('got list of decks: {}'.format(result))

In [13]:
subs = []
with open(Params.episode + ".srt") as f:
    subs = list(srt.parse(f))

In [14]:
KNOWN_WORDS = set()
with open('hsk1-5a.txt') as g:
    for word in g:
        KNOWN_WORDS.add(word.strip())
        
len(KNOWN_WORDS)

2148

In [15]:
# known_words_note_ids = invoke('findNotes', query='"deck:Bank::subs2srs::Mandarin Corner"')
# # print(type(result))
# # print('Note IDs: {}'.format(known_words_note_ids))
# known_words_note_ids += invoke('findNotes', query='"deck:Bank::subs2srs::Beauty Standards"')
# known_words_note_ids += invoke('findNotes', query='"deck:Bank::subs2srs::Chinese Stereotypes"')
# known_words_note_ids += invoke('findNotes', query='"deck:Bank::subs2srs::Livestreaming"')
# known_words_note_ids += invoke('findNotes', query='"deck:Bank::subs2srs::Lying Flat"')

known_words_note_ids = invoke('findNotes', query='"deck:Bank::subs2srs::Mandarin Corner"')
# print(type(result))
# print('Note IDs: {}'.format(known_words_note_ids))
known_words_note_ids += invoke('findNotes', query='"deck:Chinese::Mandarin Corner::Beauty Standards"')
known_words_note_ids += invoke('findNotes', query='"deck:Chinese::Mandarin Corner::Chinese Stereotypes"')
known_words_note_ids += invoke('findNotes', query='"deck:Chinese::Mandarin Corner::Livestreaming"')
known_words_note_ids += invoke('findNotes', query='"deck:Chinese::Mandarin Corner::Lying Flat"')

In [16]:
known_words_notes = invoke("notesInfo", notes=known_words_note_ids)
# known_words_notes[0]["fields"]["VocabHanzi"]["value"]
for word in known_words_notes:
    KNOWN_WORDS.add(word["fields"]["VocabHanzi"]["value"])

In [17]:
len(KNOWN_WORDS)

3100

In [18]:
NEW_WORDS = dict()

In [19]:
class WordEntry:

    def __init__(self, word, subtitle, index):
        self.word = word
        self.subtitle = subtitle
        self.index = index

In [20]:
no_def = 0
for sub in subs:
    line = sub.content.replace(" ","").strip()
    seg_list = list(jieba.cut(line, cut_all=False))
    for word in seg_list: 
        if word not in KNOWN_WORDS and word not in PUNCTUATION:
            if word in CEDICT:
                print(word, ", ".join(CEDICT[word].meanings))
            else:
                no_def += 1
                print(word)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.376 seconds.
Prefix dict has been built successfully.


,
来到 to come, to arrive
MandarinCorner
,
艾琳
,
很多
人会
联想 to associate (cognitively), to make an associative connection, mental association, word prediction and auto-complete functions of input method editing programs
14
人口 population, people
人口 population, people
大国 a power (i.e. a dominant country)
称号 name, term of address, title
,
一次 first, first time, once, (math.) linear (of degree one)
人口普查 census
,
媒体 media, esp. news media
令人担忧
新生儿 newborn baby, neonate
出生率 birthrate
下降 to decline, to drop, to fall, to go down, to decrease
手段 method, means (of doing sth), strategy, trick, CL:個|个[ge4]
,
政府 government, CL:個|个[ge4]
出台 to officially launch (a policy, program etc), to appear on stage, to appear publicly, (of a bar girl) to leave with a client
三胎
这个 this, this one
,
一期
街头 street
这个 this, this one
那么 like that, in that way, or so, so, so very much, about, in that case
,
人口 population, people
这么 so much, this much, how much?, this way, like this
还会
出生率 birthrate
下降 to decline, to drop, to

In [21]:
no_def

563

In [26]:
def add_to_anki(label):
    word = label.description
    subtitle = NEW_WORDS[word].subtitle
    index = NEW_WORDS[word].index
    seg_list = list(jieba.cut(subtitle.content, cut_all=False))
#     seg_list = [i for i in seg_list if i not in zh_hanzi.punctuation]
    start_timestamp = NEW_WORDS[word].subtitle.start.total_seconds()*1000
    end_timestamp = NEW_WORDS[word].subtitle.end.total_seconds()*1000 
    
    UniqueIdentifier = word + "|" + subtitle.content 
    SentHanzi = subtitle.content.replace(word, "<b>" + word + "</b>")
    SentPinyin = " ".join([hanzi.to_pinyin(i) for i in seg_list if i not in PUNCTUATION])
#     VocabPinyin = transcriptions.numbered_to_accented(CEDICT[word].pinyin).replace(" ", "")
    VocabPinyin = hanzi.to_pinyin(word)
    VocabDef = ", ".join(CEDICT[word].meanings) if word in CEDICT else ""
    
    if Params.is_video:
        vidcap.set(cv2.CAP_PROP_POS_MSEC, (start_timestamp))  
        success, image = vidcap.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    s = word + subtitle.content
    hash_object = hashlib.sha1(s.encode('utf-8'))
    file_stub = hash_object.hexdigest()
    
    if platform.startswith("darwin"):
        media_dir = Path("/Users/Daniel/programming/chinese/Mandarin corner/media/")
    elif platform.startswith("linux"):
        media_dir = Path("/home/daniel/programming/chinese/chinese-tools/Mandarin corner/media/")
    
    img_path = media_dir.joinpath(file_stub + ".jpg")
    audio_path = media_dir.joinpath(file_stub + "mp3")
        

#     if not Path("/home/daniel/programming/chinese/mandarin_corner/media/" + file_stub + ".jpg").exists():
    
    if Params.is_video:
        if not img_path.exists():
            im = Image.fromarray(image)
            im.save(img_path, quality=50)
    if not audio_path.exists():
        audio_clip = audio_file[max(start_timestamp - 200,0) : end_timestamp + 200]
        audio_clip.export(audio_path)

    note = {
#         "deckName": "Bank::subs2srs::" + DECK_MAP[Params.episode],
        "deckName": "Chinese::Mandarin Corner::" + DECK_MAP[Params.episode],
        "modelName": "Chinese sentences",
        "fields": {
            "UniqueIdentifier": UniqueIdentifier,
            "SentHanzi": SentHanzi,
            "SentPinyin": SentPinyin,
            "VocabHanzi": word,
            "VocabPinyin": VocabPinyin,
            "VocabDef": VocabDef,
            "Misc": DECK_MAP[Params.episode] + " - " + str(index) + " - " + str(datetime.timedelta(seconds=math.floor(start_timestamp/1000)))
        },
        "audio" : [{
            "path" : str(audio_path),
            "filename": file_stub + ".mp3",
            "fields" : [
                "SentAudio"
            ]
        }],
    }
    
    if Params.is_video:
        note["picture"] = [{
            "path" : str(img_path),
            "filename": file_stub + ".jpg",
            "fields" : [
                "Image"
            ]
        }]

    result = invoke('addNote', note=note)

In [27]:
NEW_WORDS = dict()
v_lines = []
i = 1

for sub in subs:
    line = sub.content.replace(" ","").strip()
    seg_list1 = list(jieba.cut(line, cut_all=False))
    seg_list2 = list(jieba.cut(line, cut_all=True))
    buttons1 = []
    buttons2 = []
    outs = []
    display1 = []
    display2 = []
    display1.append(widgets.Label(str(i) + "."))
    display2.append(widgets.Label("    "))
    for word in seg_list1:
        word_entry = WordEntry(word, sub, i)
        if word not in KNOWN_WORDS and word not in zh_hanzi.punctuation and word not in NEW_WORDS:
            NEW_WORDS[word] = word_entry
            button = widgets.Button(description=word_entry.word, style=dict(
                    font_size="20px"
            ))
            #             button.style.font_size = 
            out = widgets.Output()
            button.on_click(add_to_anki)
            display1.append(button)
        else:
            display1.append(widgets.Label(word))
    v_lines.append(widgets.HBox(display1))
    if seg_list2 != seg_list1:
            for word in [i for i in seg_list2 if i not in seg_list1]:
                word_entry = WordEntry(word, sub, i)
                if word not in KNOWN_WORDS and word not in PUNCTUATION and word not in NEW_WORDS:
                    NEW_WORDS[word] = word_entry
                    button = widgets.Button(description=word_entry.word)
        #             button.style.font_size = 
                    out = widgets.Output()
                    button.on_click(add_to_anki)
                    display2.append(button)
                else:
                    display2.append(widgets.Label(word))
            v_lines.append(widgets.HBox(display2))
    i += 1

widgets.VBox(v_lines)

In [28]:
stereotypes_word_ids = invoke('findNotes', query='"deck:Bank::subs2srs::Chinese Stereotypes"')
stereotypes_words_notes = invoke("notesInfo", notes=stereotypes_word_ids)
STEREOTYPE_WORDS = set()
# known_words_notes[0]["fields"]["VocabHanzi"]["value"]
for word in stereotypes_words_notes:
    STEREOTYPE_WORDS.add(word["fields"]["VocabHanzi"]["value"])

In [25]:
STEREOTYPE_WORDS

set()

In [21]:
with open('hsk5-nouns.txt') as f:
    lines = f.readlines()
    print(lines[0].split()[1].split('/')[0])
    print(lines[3])

FileNotFoundError: [Errno 2] No such file or directory: 'hsk5-nouns.txt'

In [ ]:
CEDICT['我们'].raw_line
transcriptions.numbered_to_accented(CEDICT['我们'].pinyin).replace(" ", "")
", ".join(CEDICT['我们'].meanings)